# 1. Taxonomic Profiling and Data Wrangling

## Objective
The goal of this notebook is to process the raw output reports generated by `Kraken2` for all 120 samples (60 T2D / 60 Control). We aim to consolidate the individual read counts into a single, comprehensive Taxa x Samples matrix (Abundance Matrix), which is the necessary input for statistical analysis and visualization of gut dysbiosis.

## Methodology
The process involves reading 120 individual `.report` files, filtering only for biologically meaningful ranks (Phylum, Class, Genus, Species), and joining the results using Pandas.

In [ ]:
import pandas as pd
import os
import glob

# Configuration: Directory containing the Kraken2 reports
# Ensure this matches your actual output directory from Snakemake
REPORTS_DIR = 'results/kraken'

# Define the column names for standard Kraken2 report format
KRAKEN_COLS = ['percent', 'reads', 'reads_taxo', 'rank', 'taxid', 'name']

def load_kraken_report(file_path):
    """
    Reads a single Kraken2 report file and formats it for analysis.
    
    Args:
        file_path (str): Path to the .report file.
        
    Returns:
        pd.DataFrame: Cleaned dataframe with taxa names as index and read counts as columns.
    """
    try:
        # Read the report, using latin-1 to handle potential special characters in scientific names
        df = pd.read_csv(file_path, sep='\t', header=None, names=KRAKEN_COLS, encoding='latin-1')
        
        # Extract the sample ID from the filename (e.g., 'SRR123456.report' -> 'SRR123456')
        sample_id = os.path.basename(file_path).replace('.report', '')
        
        # Filter: Keep only standard taxonomic ranks (Phylum to Species)
        # P=Phylum, C=Class, O=Order, F=Family, G=Genus, S=Species
        # We exclude 'U' (Unclassified) and 'R' (Root) for this specific downstream analysis
        df_filtered = df[df['rank'].isin(['P', 'C', 'O', 'F', 'G', 'S'])].copy()
        
        # Strip whitespace from taxa names
        df_filtered['name'] = df_filtered['name'].str.strip()
        
        # Select only the 'name' (Taxa) and 'reads' (Abundance) columns
        df_final = df_filtered[['name', 'reads']]
        
        # Rename the 'reads' column to the Sample ID (crucial for merging later)
        df_final = df_final.rename(columns={'reads': sample_id})
        
        # Set the taxa name as the index
        return df_final.set_index('name')
        
    except Exception as e:
        print(f"Error loading {file_path}: {e}")
        return None

In [ ]:
# --- Step 1: Find all report files ---
# Search for all files ending with .report in the directory
report_files = glob.glob(os.path.join(REPORTS_DIR, '*.report'))
print(f"✅ Found {len(report_files)} Kraken2 report files.")

# --- Step 2: Load and Process ---
print("⏳ Loading and merging files... This might take a moment.")

# Iterate through all files and load them into a list
all_dfs = []
for f in report_files:
    df = load_kraken_report(f)
    if df is not None:
        all_dfs.append(df)

# --- Step 3: Concatenate into a single Matrix ---
# axis=1 means we join columns (samples) side by side
# fillna(0) replaces missing values with 0 (if a species is missing in a sample, count is 0)
final_tax_table = pd.concat(all_dfs, axis=1, join='outer').fillna(0)

# --- Step 4: Review the Result ---
print("\n=== Final Taxonomy Matrix ===")
print(f"📊 Dimensions: {final_tax_table.shape[0]} Taxa (Rows) x {final_tax_table.shape[1]} Samples (Columns)")

# Display the first 5 rows to verify
display(final_tax_table.head())

## 2. Metadata Integration
To perform comparative analysis, we need to map each Run ID (`SRR...`) to its experimental group (Type 2 Diabetes vs. Healthy Control). We retrieve this metadata directly from the ENA API using the sample aliases (`bgi-T2D` vs `bgi-HT`).

In [ ]:
# Install visualization libraries directly
!pip install matplotlib seaborn

In [ ]:
# --- Corrected Metadata Mapping ---
# Logic: We downloaded 60 T2D and 60 Non-T2D.
# So, if alias has 'T2D' -> Case, otherwise -> Control.

# 1. Create the dictionary
run_to_group = {}
for index, row in meta_raw.iterrows():
    run_id = row['run_accession']
    alias = str(row['sample_alias'])
    
    if 'T2D' in alias:
        run_to_group[run_id] = 'T2D'
    else:
        # Since we pre-selected controls, anything else is a Control
        run_to_group[run_id] = 'Control'

# 2. Map to our samples
metadata = pd.DataFrame(index=final_tax_table.columns)
metadata['Group'] = metadata.index.map(run_to_group)

# 3. Verify (Should be 60 vs 60 now)
print("\n=== Corrected Sample Distribution ===")
print(metadata['Group'].value_counts())

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# 1. Data Preparation 
taxa_sums = final_tax_table.sum(axis=1)
# نأخذ أعلى 10
top_10_taxa = taxa_sums.sort_values(ascending=False).head(10)

# 2. Plotting 
plt.figure(figsize=(12, 8)) 
sns.barplot(x=top_10_taxa.values, y=top_10_taxa.index, palette="viridis")

plt.title("Top 10 Most Abundant Taxa (Across 120 Samples)", fontsize=16)
plt.xlabel("Total Reads", fontsize=14)
plt.ylabel("Taxon Name", fontsize=14)

# 3. Saving 
# dpi=300: 
# bbox_inches='tight': 
plt.savefig("top10_taxa_abundance.png", dpi=300, bbox_inches='tight')

# 4. Show
plt.show()

print("✅ Graph saved as 'top10_taxa_abundance.png'")

# 3. Conclusion & Next Steps

## Scientific Observations
1.  **Data Integrity:** Successfully processed and classified **120 metagenomic samples** (60 Type 2 Diabetes cases vs. 60 Healthy controls). The high classification rate confirms the quality of the `fastp` trimming and `Kraken2` profiling.
2.  **Taxonomic Landscape:** The community composition is dominated by the **Bacteroidetes** phylum (specifically the *Bacteroides* genus) across the dataset. This aligns with established literature linking *Bacteroides* enterotypes to long-term diets rich in protein and animal fats, often associated with metabolic disorders like T2D.
3.  **Firmicutes Ratio:** The visualization suggests a potential shift in the *Firmicutes/Bacteroidetes* ratio, a classic biomarker for dysbiosis in obesity and diabetes.

## Future Directions (Functional Analysis)
While taxonomic profiling answers "Who is there?", it does not fully explain the metabolic implications of dysbiosis. The next phase of this project involves **De Novo Assembly (MEGAHIT)** and **Gene Prediction** to answer:
* *What are they doing?*
* *Are there specific genes (e.g., butyrate production pathways) depleted in T2D patients?*